In [1]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
ex = "010"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"

In [3]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':32,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 10000,
    'early_stopping_rounds': 200,
    'verbose_eval': 200,
}

load_feature = ["../output/fe/fe001.feather",
                "../output/fe/fe002.feather",
                "../output/fe/fe003.feather",
                "../output/fe/fe004.feather",
                "../output/fe/fe005.feather",
                "../output/fe/fe006.feather",
                "../output/fe/fe007.feather",
                "../output/fe/fe009.feather",
                "../output/fe/fe010.feather",]

In [4]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [5]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-17 12:59:43,327 - INFO - logger set up


<RootLogger root (DEBUG)>

In [6]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

In [7]:
y

0       8.78
1       8.06
2       7.90
3       6.38
4       8.17
        ... 
4995    7.23
4996    6.54
4997    6.51
4998    5.92
4999    6.83
Name: Score, Length: 5000, dtype: float64

In [8]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    LOGGER.info(f'CV={score}')

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.251995	valid_1's rmse: 0.382039
[400]	training's rmse: 0.174684	valid_1's rmse: 0.373453
[600]	training's rmse: 0.130837	valid_1's rmse: 0.371519
Early stopping, best iteration is:
[589]	training's rmse: 0.132776	valid_1's rmse: 0.371418


2021-08-17 12:59:51,783 - INFO - Fold0:CV=0.37141754142681455
2021-08-17 12:59:51,786 - INFO - [fold 0] done in 8 s


Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.246809	valid_1's rmse: 0.378531
[400]	training's rmse: 0.169908	valid_1's rmse: 0.369303
[600]	training's rmse: 0.126425	valid_1's rmse: 0.366294
[800]	training's rmse: 0.0979859	valid_1's rmse: 0.365213
[1000]	training's rmse: 0.0779495	valid_1's rmse: 0.364173
Early stopping, best iteration is:
[938]	training's rmse: 0.083481	valid_1's rmse: 0.364138


2021-08-17 13:00:06,517 - INFO - Fold1:CV=0.3641379767189375
2021-08-17 13:00:06,521 - INFO - [fold 1] done in 15 s


Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.250464	valid_1's rmse: 0.397644
[400]	training's rmse: 0.174579	valid_1's rmse: 0.387496
[600]	training's rmse: 0.131018	valid_1's rmse: 0.385412
[800]	training's rmse: 0.101997	valid_1's rmse: 0.384032
[1000]	training's rmse: 0.0815932	valid_1's rmse: 0.383886
Early stopping, best iteration is:
[848]	training's rmse: 0.0964757	valid_1's rmse: 0.383743


2021-08-17 13:00:17,711 - INFO - Fold2:CV=0.3837433848029237
2021-08-17 13:00:17,726 - INFO - [fold 2] done in 11 s


Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.25139	valid_1's rmse: 0.377194
[400]	training's rmse: 0.174428	valid_1's rmse: 0.367891
[600]	training's rmse: 0.130378	valid_1's rmse: 0.365688
[800]	training's rmse: 0.10121	valid_1's rmse: 0.365674
Early stopping, best iteration is:
[648]	training's rmse: 0.122331	valid_1's rmse: 0.365059


2021-08-17 13:00:25,378 - INFO - Fold3:CV=0.365059223601556
2021-08-17 13:00:25,385 - INFO - [fold 3] done in 8 s


Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.243733	valid_1's rmse: 0.411023
[400]	training's rmse: 0.167338	valid_1's rmse: 0.399656
[600]	training's rmse: 0.12326	valid_1's rmse: 0.396291
[800]	training's rmse: 0.0947934	valid_1's rmse: 0.396279
[1000]	training's rmse: 0.0746844	valid_1's rmse: 0.396068
[1200]	training's rmse: 0.0603322	valid_1's rmse: 0.395955
Early stopping, best iteration is:
[1139]	training's rmse: 0.064292	valid_1's rmse: 0.395752


2021-08-17 13:00:37,475 - INFO - Fold4:CV=0.39575154461123424
2021-08-17 13:00:37,485 - INFO - [fold 4] done in 12 s
2021-08-17 13:00:37,498 - INFO - CV=0.3762164173311033
2021-08-17 13:00:37,500 - INFO - [lightgbm] done in 54 s


In [9]:
y_test_sub =  np.mean(y_test,axis=0)
sub = pd.read_csv(SUB_PATH)

In [10]:
sub["Score"] = y_test_sub
sub.to_csv(SAVE_TEST_SUB_PATH,index=False)

In [11]:
feature_importances.groupby(by="feature").mean().sort_values(by="gain",ascending=False).iloc[:10]

,gain,split,fold
feature,,,
Favorites,5556.654376,661.2,2.0
user_id_emb_2,3287.926015,261.4,2.0
Dropped,1791.573069,1403.8,2.0
user_id_emb_19,1594.092393,366.8,2.0
anime_id_rate_std,984.276060,1653.2,2.0
Duration,861.136628,799.8,2.0
user_id_emb_6,791.786511,178.8,2.0
On-Hold,788.124635,900.4,2.0
user_id_emb_0,532.564331,649.6,2.0
